In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import ta
import numpy as np
import datetime
from IPython.display import clear_output
from utilities.data_manager import ExchangeDataManager
from utilities.custom_indicators import get_n_columns
from utilities.bt_analysis import get_metrics, backtest_analysis
from utilities.plot_analysis import plot_equity_vs_asset, plot_futur_simulations, plot_bar_by_month
import nest_asyncio
nest_asyncio.apply()

In [ ]:
class Strategy():
    def __init__(
        self,
        df_list,
        oldest_pair,
        type=["long"],
        params={},
    ):
        self.df_list = df_list
        self.oldest_pair = oldest_pair
        self.use_long = True if "long" in type else False
        self.use_short = True if "short" in type else False
        self.params = params

        
    def populate_indicators(self):
        params = self.params
        for pair in self.df_list:
            df = self.df_list[pair]
            df.drop(
                columns=df.columns.difference(['open','high','low','close','volume']), 
                inplace=True
            )
            
            # -- Populate indicators --    
            df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
            df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
            df['mrat'] = df['fast_ma'] / df['slow_ma']
        
            self.df_list[pair] = df
                
        return self.df_list[self.oldest_pair]
        
    def run_backtest(self, initial_wallet=1000, leverage=1):
        params = self.params
        df_ini = self.df_list[self.oldest_pair][:]
        wallet = initial_wallet
        maker_fee = 0.0002
        taker_fee = 0.0006
        trades = []
        days = []
        current_day = 0
        previous_day = 0
        current_period = 0
        previous_period = 0
        current_positions = {}
        
        for index, ini_row in df_ini.iterrows():
            # -- Add daily report --
            current_day = index.day
            current_period = index.week
            if previous_day != current_day:
                temp_wallet = wallet
                for pair in current_positions:
                    row = self.df_list[pair].loc[index]
                    if current_positions[pair]['side'] == "LONG":
                        close_price = row['open']
                        trade_result = (close_price - current_positions[pair]['price']) / current_positions[pair]['price']
                        close_size = current_positions[pair]['size'] + current_positions[pair]['size']  * trade_result
                        fee = close_size * taker_fee
                        temp_wallet += close_size - current_positions[pair]['size'] - fee
                    elif current_positions[pair]['side'] == "SHORT":
                        close_price = row['open']
                        trade_result = (current_positions[pair]['price'] - close_price) / current_positions[pair]['price']
                        close_size = current_positions[pair]['size'] + current_positions[pair]['size']  * trade_result
                        fee = close_size * taker_fee
                        temp_wallet += close_size - current_positions[pair]['size'] - fee
                    
                days.append({
                    "day":str(index.year)+"-"+str(index.month)+"-"+str(index.day),
                    "wallet":temp_wallet,
                    "price":ini_row['open'],
                    "long_exposition":0,
                    "short_exposition":0,
                    "risk":0,
                })
            previous_day = current_day


            if previous_period != current_period:
                mrat_dict = {}
                # --- Close all positions ---
                for pair in current_positions.copy():
                    position = current_positions[pair]
                    row = self.df_list[pair].loc[index]
                    if position["side"] == "LONG":
                        close_price = row['close']
                        trade_result = (close_price - position['price']) / position['price']
                        close_size = position['size'] + position['size'] * trade_result
                        fee = close_size * taker_fee
                        wallet += close_size - position['size'] - fee
                        trades.append({
                            "pair": pair,
                            "open_date": position['date'],
                            "close_date": index,
                            "position": position['side'],
                            "open_reason": position['reason'],
                            "close_reason": "Market",
                            "open_price": position['price'],
                            "close_price": close_price,
                            "open_fee": position['fee'],
                            "close_fee": fee,
                            "open_trade_size": position['size'],
                            "close_trade_size": close_size,
                            "wallet": wallet,
                        })
                        del current_positions[pair]
                    elif position["side"] == "SHORT":
                        close_price = row['close']
                        trade_result = (position['price'] - close_price) / position['price']
                        close_size = position['size'] + position['size'] * trade_result
                        fee = close_size * taker_fee
                        wallet += close_size - position['size'] - fee
                        trades.append({
                            "pair": pair,
                            "open_date": position['date'],
                            "close_date": index,
                            "position": position['side'],
                            "open_reason": position['reason'],
                            "close_reason": "Market",
                            "open_price": position['price'],
                            "close_price": close_price,
                            "open_fee": position['fee'],
                            "close_fee": fee,
                            "open_trade_size": position['size'],
                            "close_trade_size": close_size,
                            "wallet": wallet,
                        })
                        del current_positions[pair]

                for pair in self.df_list:
                    if index in self.df_list[pair].index:
                        row = self.df_list[pair].loc[index]
                        # --- Add MRAT to dict ---
                        if np.isnan(row['mrat']) == False:
                            mrat_dict[pair] = row['mrat']
                        
                # --- Check for open new positions ---
                mrat_values = list(mrat_dict.values())
                if len(mrat_values) < 2:
                    previous_period = current_period
                    continue
                mean = np.mean(mrat_values)
                sigma = np.std(mrat_values)
                threshold_high = 1 + (params["sigma_multiplier"] * sigma)
                threshold_low = 1 - (params["sigma_multiplier"] * sigma)
                long_list = [k for k,v in mrat_dict.items() if v > threshold_high and self.use_long]
                short_list = [k for k,v in mrat_dict.items() if v < threshold_low and self.use_short]
                for pair in long_list:
                    if pair not in current_positions:
                        row = self.df_list[pair].loc[index]
                        open_price = row["close"]
                        pos_size = ((0.5 / len(long_list)) * wallet * leverage)
                        fee = pos_size * taker_fee
                        pos_size -= fee
                        wallet -= fee
                        current_positions[pair] = {
                            "size": pos_size,
                            "date": index,
                            "price": open_price,
                            "fee":fee,
                            "reason": "Market",
                            "side": "LONG",
                        }
                for pair in short_list:
                    if pair not in current_positions:
                        row = self.df_list[pair].loc[index]
                        open_price = row["close"]
                        pos_size = ((0.5 / len(short_list)) * wallet * leverage)
                        fee = pos_size * taker_fee
                        pos_size -= fee
                        wallet -= fee
                        current_positions[pair] = {
                            "size": pos_size,
                            "date": index,
                            "price": open_price,
                            "fee":fee,
                            "reason": "Market",
                            "side": "SHORT",
                        }

            previous_period = current_period  
            
                       
                        
        df_days = pd.DataFrame(days)
        df_days['day'] = pd.to_datetime(df_days['day'])
        df_days = df_days.set_index(df_days['day'])

        df_trades = pd.DataFrame(trades)
        df_trades['open_date'] = pd.to_datetime(df_trades['open_date'])
        df_trades = df_trades.set_index(df_trades['open_date'])   
        
        return get_metrics(df_trades, df_days) | {
            "wallet": wallet,
            "trades": df_trades,
            "days": df_days
        }      

In [ ]:
params = {
    "fast_ma": 10,
    "slow_ma": 100,
    "sigma_multiplier": 0.5,
}

pair_list = ['AAVE/USDT', 'APE/USDT', 'APT/USDT', 'AVAX/USDT', 'AXS/USDT', 'C98/USDT', 'CRV/USDT', 'DOGE/USDT', 'DOT/USDT', 'DYDX/USDT', 'ETH/USDT', 'FIL/USDT', 'FTM/USDT', 'BNB/USDT', 'GALA/USDT', 'GMT/USDT', 'GRT/USDT', 'KNC/USDT', 'KSM/USDT', 'LRC/USDT', 'MANA/USDT', 'MASK/USDT', 'MATIC/USDT', 'NEAR/USDT', 'ONE/USDT', 'OP/USDT', 'SAND/USDT', 'SHIB/USDT', 'SOL/USDT', 'STG/USDT', 'WAVES/USDT', 'YFI/USDT', 'WOO/USDT', 'BTC/USDT', 'EGLD/USDT', 'ETC/USDT', 'JASMY/USDT', 'ROSE/USDT']
# pair_list = ['BTC/USDT', 'EGLD/USDT', 'ETH/USDT']
exchange_name = "binance"
tf = '4h'
oldest_pair = "BTC/USDT"

exchange = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download="../database/exchanges"
)

df_list = {}
for pair in pair_list:
    df = exchange.load_data(pair, tf)
    df_list[pair] = df.loc[:]

print("Data load 100%")
df_list[oldest_pair]

In [ ]:
strat = Strategy(
    df_list=df_list,
    oldest_pair=oldest_pair,
    type=["long", "short"],
    params=params,
)

strat.populate_indicators()
bt_result = strat.run_backtest(initial_wallet=1000, leverage=1)

df_trades, df_days = backtest_analysis(
    trades=bt_result['trades'], 
    days=bt_result['days'],
    general_info=True,
    trades_info=True,
    days_info=True,
    long_short_info=True,
    entry_exit_info=True,
    exposition_info=True,
    pair_info=True,
    indepedant_trade=True
)

In [ ]:
# df_trades.to_excel("trades.xlsx")
df_trades

In [ ]:
plot_equity_vs_asset(df_days=df_days.loc[:])

In [ ]:
plot_bar_by_month(df_days=df_days)

In [ ]:
df_trades.sort_values(by=["trade_result_pct"])